In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import time

print(torch.cuda.is_available())  # Should print True if GPU is available
print(torch.cuda.device_count())  # Number of GPUs available
print(torch.__version__)  # Should include '+cu118' or another CUDA version
#print(torch.cuda.get_device_name(0))  # Name of the first GPU
#print(torch.cuda.current_device())  # Should return 0 if a GPU is being used

In [ ]:
# Define the feedforward neural network
class FeedforwardMLP(nn.Module):
    def __init__(self):
        super(FeedforwardMLP, self).__init__()
        self.fc1 = nn.Linear(32 * 32 * 3, 4096)
        self.bn1 = nn.BatchNorm1d(4096)
        self.fc2 = nn.Linear(4096, 2048)
        self.bn2 = nn.BatchNorm1d(2048)
        self.fc3 = nn.Linear(2048, 1024)
        self.bn3 = nn.BatchNorm1d(1024)
        self.fc4 = nn.Linear(1024, 512)
        self.bn4 = nn.BatchNorm1d(512)
        self.fc5 = nn.Linear(512, 256)
        self.bn5 = nn.BatchNorm1d(256)
        self.fc6 = nn.Linear(256, 128)
        self.bn6 = nn.BatchNorm1d(128)
        self.fc7 = nn.Linear(128, 64)
        self.bn7 = nn.BatchNorm1d(64)
        self.fc8 = nn.Linear(64, 10)  # Output layer
        self.dropout = nn.Dropout(0.5)  # Stronger dropout
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten input
        x = self.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)
        x = self.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)
        x = self.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)
        x = self.relu(self.bn4(self.fc4(x)))
        x = self.dropout(x)
        x = self.relu(self.bn5(self.fc5(x)))
        x = self.dropout(x)
        x = self.relu(self.bn6(self.fc6(x)))
        x = self.dropout(x)
        x = self.relu(self.bn7(self.fc7(x)))
        x = self.dropout(x)
        x = self.fc8(x)  # No activation for softmax (CrossEntropyLoss handles it)
        return x

In [ ]:
# Hyperparameters
#input_size = 784  # 28x28 images flattened
#hidden_size = 128
#num_classes = 10
num_epochs = 200
batch_size = 512
learning_rate = 0.001

In [ ]:
# Load the MNIST dataset
train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(policy=transforms.AutoAugmentPolicy.CIFAR10),  # Stronger augmentation
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=test_transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


In [ ]:
# Define model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = FeedforwardMLP().to(device)
print(next(model.parameters()).device)


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-4)  # AdamW for better training
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)  # Smooth LR decay

# Training loop
def train():
    model.train()
    for epoch in range(num_epochs):
        for batch_idx, (images, labels) in enumerate(train_loader):

            images, labels = images.to(device), labels.to(device)  # Move to GPU
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 100 == 0:
                print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        scheduler.step()  # Adjust learning rate

# Evaluate accuracy
def test():
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)  # Move to GPU
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Test Accuracy: {100 * correct / total:.2f}%")


if __name__ == "__main__":
    import time

    # Measure training time
    start_time = time.time()
    train()
    end_time = time.time()

    training_time = end_time - start_time
    print(f"Training time: {training_time:.2f} seconds")

    # Measure inference time
    start_time = time.time()
    test()
    end_time = time.time()

    inference_time = end_time - start_time
    print(f"Inference time: {inference_time:.2f} seconds for the entire test set")

    per_sample_inference_time = inference_time / len(test_dataset)
    print(f"Inference time per sample: {per_sample_inference_time:.6f} seconds")



In [ ]:


# Save the model to a file
model_path = "feedforward_MLP.pth"
torch.save(model.state_dict(), model_path)

# Measure the size of the model file
model_size = os.path.getsize(model_path) / (1024 * 1024)  # Convert bytes to MB
print(f"Model size: {model_size:.2f} MB")